# 환경 셋업

In [ ]:
# 시스템 업데이트
!apt-get update -qq > /dev/null 2>&1

# 기존 Node.js 제거 (있다면)
!apt-get remove -y nodejs npm > /dev/null 2>&1

# NodeSource 저장소 추가하여 Node.js 20 설치
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash - > /dev/null 2>&1
!apt-get install -y nodejs > /dev/null 2>&1

# pyngrok 설치
!pip install -q pyngrok > /dev/null 2>&1

# 중요한 정보만 출력
!echo "Node.js 버전: $(node --version)"
!echo "NPM 버전: $(npm --version)"

# 라이브 스트리밍 데모 저장소 복제
!git clone https://github.com/de-id/live-streaming-demo.git > /dev/null 2>&1

# 프로젝트 디렉토리로 이동 후 설치
%cd /content/live-streaming-demo
!npm install --silent > /dev/null 2>&1

# 셀 실행 후 이 코드를 추가
from IPython.display import clear_output
clear_output(wait=True)
print("설치가 완료되었습니다.")

설치가 완료되었습니다.


In [ ]:
# 파이썬 라이브러리 임포트 및 API 키 로딩
import openai
from openai import OpenAI
import requests
import google.colab.userdata as userdata
import base64
import time
import subprocess
from pyngrok import ngrok
import shutil
import re

# OpenAI API 키 가져오기
# openai_api_key = userdata.get("OPENAI_API_KEY") # 환경변수 이용하는 경우
openai_api_key = "sk-onhr1ZcXfiEEMpavrFrWxZ7Zpx7qdS0ajYe-8xQsCYT3BlbkFJqGOch3bX_2OBss9UzN5n4tXg3-DQYXy1ZP-B8VSRYA"

# D-ID API 키 가져오기 (raw 상태)
# raw_api_key = userdata.get("DID_API_KEY") # 환경변수 이용하는 경우
raw_api_key = "c2RrcGFya2ZvcmJpQGdtYWlsLmNvbQ:9ssVAjhSBZ364z34ooqEX"

# D-ID API 키 Base64 인코딩
API_KEY = base64.b64encode(raw_api_key.encode()).decode()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=openai_api_key)

# 사용할 이미지 URL 예시
SOURCE_URL = "https://d-id-public-bucket.s3.us-west-2.amazonaws.com/alice.jpg"
API_KEY, SOURCE_URL, raw_api_key

('YzJScmNHRnlhMlp2Y21KcFFHZHRZV2xzTG1OdmJROjlzc1ZBamhTQlozNjR6MzRvb3FFWA==',
 'https://d-id-public-bucket.s3.us-west-2.amazonaws.com/alice.jpg',
 'c2RrcGFya2ZvcmJpQGdtYWlsLmNvbQ:9ssVAjhSBZ364z34ooqEX')

In [ ]:
import json

# JSON 내용을 딕셔너리 형태로 정의
data = {
    "key": API_KEY,
    "url": "https://api.d-id.com",
    "websocketUrl": "wss://ws-api.d-id.com",
    "service": "talks",
    "elevenlabsKey": ""
}

with open('api.json', 'w') as f:
    json.dump(data, f)

In [ ]:
# 기존 3000포트 사용하고 있는 경우 종료
!fuser -k 3000/tcp

In [ ]:
# 노드서버 실행
node_process = subprocess.Popen(
    ['node', 'app.js'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)

# 상태 확인
if node_process.poll() is not None:
    stderr_output = node_process.stderr.read().decode()
    print("Node 실행 실패:\n", stderr_output)
else:
    print("Node 서버가 정상적으로 실행 중입니다.")

node_process.poll()

Node 서버가 정상적으로 실행 중입니다.


In [ ]:
# 내부적으로 ngrok 프로세스를 중단하고 터널 종료

# 현재 열려 있는 모든 터널 종료
ngrok.kill()

In [ ]:
# ngrok 프로세스 실행
# https://ngrok.com/ 접속 깃허브나 구글계정을 이용하여 로그인
# *** 본인 키로 반드시 바꿀 것 - 1세션만 가능 ****
ngrok.set_auth_token("2z9tlMkfxPzkscG0OZw2gkSbpGs_7uM1hYqsY6omqPeV7K3sS")

# ─── 6) 포트 3000을 외부에 노출 ───
public_url = ngrok.connect(3000, "http")
ngrok_url = public_url.public_url  # 여기에 현재 주소 저장   #수정 부분

print("접속 링크:", public_url.public_url + "/agents")

접속 링크: https://0b8095e3d616.ngrok-free.app/agents


# 수정할 파일 확인

In [ ]:
import os
os.getcwd()

'/content/live-streaming-demo'

# 버전 선택

In [ ]:
# 1단계: JS 파일 백업
shutil.copy('agents-client-api.js', 'agents-client-api.js.backup')

# 2단계: HTML 파일 백업
shutil.copy('index-agents.html', 'index-agents.html.backup')

'index-agents.html.backup'

## 상태창과 아바타, 대화자막있는 버전

In [ ]:
import shutil
import re

# 1단계: 백업 복구
shutil.copy('agents-client-api.js.backup', 'agents-client-api.js')
shutil.copy('index-agents.html.backup', 'index-agents.html')

# 2단계: 원본 읽기
with open('agents-client-api.js', 'r', encoding='utf-8') as f:
    js_content = f.read()

# 3단계: OpenAI API 키
openai_api_key = "sk-onhr1ZcXfiEEMpavrFrWxZ7Zpx7qdS0ajYe-8xQsCYT3BlbkFJqGOch3bX_2OBss9UzN5n4tXg3-DQYXy1ZP-B8VSRYA"

# 4단계: 완전 자동화 음성 대화 시스템 코드
improved_stt_code = f"""// 상태 메시지창 생성 및 관리
function createStatusWindow() {{
  const existingStatus = document.getElementById('custom-status-window');
  if (existingStatus) {{
    existingStatus.remove();
  }}

  const statusWindow = document.createElement('div');
  statusWindow.id = 'custom-status-window';
  statusWindow.style.position = 'fixed';
  statusWindow.style.top = '20px';
  statusWindow.style.right = '20px';
  statusWindow.style.width = '280px';
  statusWindow.style.background = 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)';
  statusWindow.style.color = 'white';
  statusWindow.style.padding = '12px';
  statusWindow.style.borderRadius = '10px';
  statusWindow.style.boxShadow = '0 4px 15px rgba(0,0,0,0.2)';
  statusWindow.style.zIndex = '10000';
  statusWindow.style.fontFamily = 'Segoe UI, Tahoma, Geneva, Verdana, sans-serif';
  statusWindow.style.fontSize = '13px';
  statusWindow.style.lineHeight = '1.4';
  statusWindow.style.border = '2px solid rgba(255,255,255,0.2)';

  const statusTitle = document.createElement('div');
  statusTitle.style.fontWeight = 'bold';
  statusTitle.style.fontSize = '15px';
  statusTitle.style.marginBottom = '8px';
  statusTitle.style.textAlign = 'center';
  statusTitle.style.borderBottom = '1px solid rgba(255,255,255,0.3)';
  statusTitle.style.paddingBottom = '6px';
  statusTitle.textContent = '🤖 Emma AI 상태';

  const statusContent = document.createElement('div');
  statusContent.id = 'status-content';
  statusContent.style.minHeight = '40px';
  statusContent.style.display = 'flex';
  statusContent.style.alignItems = 'center';
  statusContent.style.justifyContent = 'center';

  statusWindow.appendChild(statusTitle);
  statusWindow.appendChild(statusContent);
  document.body.appendChild(statusWindow);

  return statusContent;
}}

function updateStatus(message, isLoading = false) {{
  const statusContent = document.getElementById('status-content');
  if (!statusContent) return;

  if (isLoading) {{
    const spinnerHtml = '<span style="display:inline-block;width:12px;height:12px;border:2px solid rgba(255,255,255,0.3);border-top:2px solid white;border-radius:50%;animation:spin 1s linear infinite;margin-right:8px;"></span>';
    const styleHtml = '<style>@keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}</style>';
    statusContent.innerHTML = '<div style="text-align: center;">' + spinnerHtml + message + '</div>' + styleHtml;
  }} else {{
    statusContent.innerHTML = '<div style="text-align: center;">' + message + '</div>';
  }}
}}

// 자막 시스템
function createSubtitleDiv() {{
  let subtitleDiv = document.getElementById('subtitle');
  if (!subtitleDiv) {{
    subtitleDiv = document.createElement('div');
    subtitleDiv.id = 'subtitle';
    subtitleDiv.style.cssText = `
      position: fixed;
      bottom: 15%;
      left: 50%;
      transform: translateX(-50%);
      background: rgba(0, 0, 0, 0.7);
      color: white;
      padding: 12px 20px;
      border-radius: 15px;
      font-size: 18px;
      max-width: 80%;
      text-align: center;
      display: none;
      z-index: 9999;
      font-family: 'Segoe UI', sans-serif;
      font-weight: 500;
      box-shadow: 0 4px 12px rgba(0,0,0,0.3);
    `;
    document.body.appendChild(subtitleDiv);
  }}
  return subtitleDiv;
}}

function showSubtitle(text, isUser = false) {{
  const subtitleDiv = createSubtitleDiv();
  if (isUser) {{
    subtitleDiv.style.background = 'rgba(52, 152, 219, 0.8)';
    subtitleDiv.textContent = `👤 ${{text}}`;
  }} else {{
    subtitleDiv.style.background = 'rgba(155, 89, 182, 0.8)';
    subtitleDiv.textContent = `🤖 ${{text}}`;
  }}
  subtitleDiv.style.display = 'block';
}}

function hideSubtitle() {{
  const subtitleDiv = document.getElementById('subtitle');
  if (subtitleDiv) {{
    subtitleDiv.style.display = 'none';
  }}
}}

// OpenAI 직접 호출
async function callOpenAI(message) {{
  updateStatus('AI가 답변을 생각하는 중...', true);

  const response = await fetch('https://api.openai.com/v1/chat/completions', {{
    method: 'POST',
    headers: {{
      'Authorization': 'Bearer {openai_api_key}',
      'Content-Type': 'application/json'
    }},
    body: JSON.stringify({{
      model: 'gpt-4o-mini',
      messages: [{{
        role: 'system',
        content: '당신은 Emma입니다. 항상 한국어로 대답하세요. 답변 시 *(별표)나 이모티콘 마크업 기능을 사용하여 답변하지말아주세요. 그리고 간략하게 답변주세요.'
      }}, {{
        role: 'user',
        content: message
      }}]
    }})
  }});
  const data = await response.json();
  updateStatus('음성으로 답변하는 중...', true);
  return data.choices[0].message.content;
}}

// STT 관련 변수들
let recognition = null;
let finalTranscript = '';
let silenceTimer = null;
let isListening = false;
let isTTSSpeaking = false;

// STT 시작 함수
function startSTT() {{
  if (isListening || isTTSSpeaking) return;

  isListening = true;
  updateStatus('🎤 음성 인식 중...', true);
  console.log("STT 시작됨");

  recognition = new (window.SpeechRecognition || window.webkitSpeechRecognition)();
  recognition.lang = 'ko-KR';
  recognition.continuous = true;
  recognition.interimResults = true;

  recognition.onresult = (event) => {{
    let interimTranscript = '';
    finalTranscript = '';

    for (let i = 0; i < event.results.length; i++) {{
      const transcript = event.results[i][0].transcript;
      if (event.results[i].isFinal) {{
        finalTranscript += transcript;
      }} else {{
        interimTranscript += transcript;
      }}
    }}

    // 실시간 자막 표시 (사용자 음성)
    const currentText = finalTranscript + interimTranscript;
    if (currentText.trim()) {{
      showSubtitle(currentText, true);
    }}

    // 침묵 감지 타이머
    clearTimeout(silenceTimer);
    silenceTimer = setTimeout(() => {{
      console.log("침묵 감지, STT 종료");
      recognition.stop();
    }}, 2000);
  }};

  recognition.onend = async () => {{
    console.log("STT 종료됨");
    isListening = false;
    clearTimeout(silenceTimer);

    const userInput = finalTranscript.trim();
    if (!userInput) {{
      console.log("입력 없음, 1초 후 STT 재시작");
      hideSubtitle();
      updateStatus('✅ 음성 대기 중', false);
      setTimeout(() => startSTT(), 1000);
      return;
    }}

    // 사용자 입력 확정 자막
    showSubtitle(userInput, true);

    // TTS 시작 표시
    isTTSSpeaking = true;

    try {{
      // AI 응답 받기
      const aiResponse = await callOpenAI(userInput);

      // Emma 응답 자막 표시
      showSubtitle(aiResponse, false);

      // DID API로 TTS 요청
      await fetchWithRetries(`${{DID_API.url}}/talks/streams/${{streamId}}`, {{
        method: 'POST',
        headers: {{
          Authorization: `Basic ${{DID_API.key}}`,
          'Content-Type': 'application/json',
        }},
        body: JSON.stringify({{
          script: {{
            type: 'text',
            input: aiResponse,
            provider: {{ type: 'microsoft', voice_id: 'ko-KR-SunHiNeural' }},
          }},
          session_id: sessionId,
        }}),
      }});

    }} catch (error) {{
      console.error('AI 응답 오류:', error);
      isTTSSpeaking = false;
      showSubtitle('죄송합니다. 오류가 발생했습니다.', false);
      setTimeout(() => {{
        hideSubtitle();
        updateStatus('✅ 음성 대기 중', false);
        startSTT();
      }}, 2000);
    }}

    finalTranscript = '';
  }};

  recognition.onerror = (event) => {{
    console.error('STT 오류:', event.error);
    isListening = false;
    clearTimeout(silenceTimer);

    hideSubtitle();
    updateStatus('❌ 음성 인식 오류', false);

    // 오류 발생시 1초 후 재시작
    setTimeout(() => {{
      console.log("STT 오류 후 재시작");
      updateStatus('✅ 음성 대기 중', false);
      startSTT();
    }}, 1000);
  }};

  try {{
    recognition.start();
  }} catch (error) {{
    console.error('STT 시작 실패:', error);
    isListening = false;
    updateStatus('❌ 음성 인식 시작 실패', false);
    setTimeout(() => {{
      updateStatus('✅ 음성 대기 중', false);
      startSTT();
    }}, 1000);
  }}
}}

// 초기 STT 시작 함수
function initializeSTT() {{
  console.log("초기 STT 시작");
  updateStatus('✅ 음성 대기 중', false);
  setTimeout(() => startSTT(), 1000);
}}

// UI 요소들 숨기기 함수
function hideUIElements() {{
  // 메인 status div 숨기기
  const statusDiv = document.getElementById('status');
  if (statusDiv) {{
    statusDiv.style.display = 'none';
    console.log('Main status div 숨김');
  }}

  // Chat History 숨기기
  const chatDiv = document.querySelector('.chat');
  if (chatDiv) {{
    chatDiv.style.display = 'none';
    console.log('Chat history 숨김');
  }}

  // Type your message 영역 숨기기
  const messageH3 = document.querySelector('h3');
  if (messageH3 && messageH3.textContent.includes('Type your message here')) {{
    if (messageH3.parentElement) {{
      messageH3.parentElement.style.display = 'none';
    }}
    console.log('Message input section 숨김');
  }}

  // textArea 숨기기
  const textarea = document.getElementById('textArea');
  if (textarea) {{
    textarea.style.display = 'none';
  }}

  // 모든 버튼 숨기기
  const allButtons = document.querySelectorAll('button');
  allButtons.forEach(btn => {{
    btn.style.display = 'none';
  }});

  // 모든 라벨과 상태 정보 숨기기
  const labels = document.querySelectorAll('label');
  labels.forEach(label => {{
    label.style.display = 'none';
  }});

  console.log('UI 요소들 완전 숨김 처리 완료');
}}

// 자동 연결 및 시작 함수
async function autoConnectAndStart() {{
  updateStatus('시스템 초기화 중...', true);
  console.log('자동 연결 시작...');

  try {{
    // 1. agents 버튼 클릭
    const agentsBtn = document.getElementById('agents-button');
    if (agentsBtn && !agentsBtn.disabled) {{
      updateStatus('Emma AI 에이전트 생성 중...', true);
      agentsBtn.click();
      console.log('Agents 생성 중...');

      // agents 완료까지 대기
      await waitForElementEnabled('connect-button', 60000);
      console.log('Agents 생성 완료');

      // Knowledge 처리 완료 대기
      await new Promise(resolve => setTimeout(resolve, 5000));

      // 2. connect 버튼 적극적으로 클릭
      updateStatus('Emma와 연결하는 중...', true);

      let connectSuccess = false;
      for (let attempt = 1; attempt <= 5; attempt++) {{
        console.log(`Connect 시도 ${{attempt}}/5`);

        const connectBtn = document.getElementById('connect-button');
        if (connectBtn) {{
          connectBtn.disabled = false;
          connectBtn.click();

          console.log(`Connect 버튼 클릭 완료 (시도 ${{attempt}})`);

          try {{
            await waitForConnection(15000);
            connectSuccess = true;
            console.log('연결 성공!');
            break;
          }} catch (error) {{
            console.log(`연결 시도 ${{attempt}} 실패:`, error.message);
            if (attempt < 5) {{
              await new Promise(resolve => setTimeout(resolve, 3000));
            }}
          }}
        }} else {{
          console.error('Connect 버튼을 찾을 수 없음');
          break;
        }}
      }}

      if (connectSuccess) {{
        updateStatus('✅ 연결 완료! 음성 대화 시작', false);
        console.log('자동 연결 완료!');
        return true;
      }} else {{
        updateStatus('❌ 연결 실패', false);
        console.error('모든 연결 시도 실패');
        return false;
      }}
    }} else {{
      console.error('Agents 버튼을 찾을 수 없음');
      updateStatus('❌ Agents 버튼 오류', false);
      return false;
    }}
  }} catch (error) {{
    console.error('자동 연결 중 오류:', error);
    updateStatus('❌ 자동연결 실패', false);
    return false;
  }}
}}

// 버튼이 활성화될 때까지 대기하는 함수
function waitForElementEnabled(elementId, timeout = 30000) {{
  return new Promise((resolve, reject) => {{
    const startTime = Date.now();

    function checkElement() {{
      const element = document.getElementById(elementId);
      if (element && !element.disabled) {{
        resolve(element);
      }} else {{
        const elapsed = Math.floor((Date.now() - startTime) / 1000);
        if (elapsed % 5 === 0 && elapsed > 0) {{
          updateStatus(`${{elementId}} 활성화 대기 중... (${{elapsed}}초)`, true);
        }}

        if (Date.now() - startTime > timeout) {{
          reject(new Error(`Timeout waiting for ${{elementId}}`));
        }} else {{
          setTimeout(checkElement, 500);
        }}
      }}
    }}

    checkElement();
  }});
}}

// 연결 완료까지 대기하는 함수
function waitForConnection(timeout = 30000) {{
  return new Promise((resolve, reject) => {{
    const startTime = Date.now();

    function checkConnection() {{
      const connectionChecks = [
        () => {{
          const isValid = typeof streamId !== 'undefined' && streamId !== null && streamId !== '';
          if (isValid) console.log('StreamId 확인됨:', streamId);
          return isValid;
        }},

        () => {{
          const videoEl = document.getElementById('video-element');
          const hasStream = videoEl && videoEl.src &&
                           !videoEl.src.includes('emma_idle.mp4') &&
                           videoEl.src.includes('blob:');
          if (hasStream) console.log('Video stream 확인됨:', videoEl.src);
          return hasStream;
        }}
      ];

      const isConnected = connectionChecks.some(check => {{
        try {{
          return check();
        }} catch (e) {{
          return false;
        }}
      }});

      if (isConnected) {{
        console.log('연결 상태 확인됨');

        if (typeof streamId === 'undefined' || streamId === null || streamId === '') {{
          console.log('StreamId 생성 대기 중...');
          setTimeout(checkConnection, 2000);
          return;
        }}

        resolve();
      }} else {{
        const elapsed = Math.floor((Date.now() - startTime) / 1000);
        if (elapsed % 3 === 0 && elapsed > 0) {{
          console.log(`연결 대기 중... ${{elapsed}}초`);
          updateStatus(`연결 중... (${{elapsed}}초)`, true);
        }}

        if (Date.now() - startTime > timeout) {{
          reject(new Error('Connection timeout'));
        }} else {{
          setTimeout(checkConnection, 1000);
        }}
      }}
    }}

    checkConnection();
  }});
}}

// 페이지 로드 시 자동 실행
document.addEventListener('DOMContentLoaded', () => {{
  setTimeout(async () => {{
    createStatusWindow();
    updateStatus('시스템 초기화 중...', true);

    // 먼저 자동 연결 시도
    const success = await autoConnectAndStart();

    // 연결 완료 후 UI 숨김
    setTimeout(() => {{
      hideUIElements();
    }}, 2000);

    if (success) {{
      // STT 초기화
      setTimeout(() => {{
        initializeSTT();
      }}, 3000);
    }} else {{
      updateStatus('❌ 연결 실패 - 새로고침하세요', false);
    }}
  }}, 2000);
}});

"""

# 5단계: Agent 설정 (간단하게)
agent_pattern = r'const createAgent = await axios\.post\(\'/agents\', \{[\s\S]*?\}\);'
simple_agent = '''const createAgent = await axios.post('/agents', {
    presenter: {
      type: 'talk',
      voice: {
        type: 'microsoft',
        voice_id: 'ko-KR-SunHiNeural',
      },
      thumbnail: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
      source_url: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
    },
    preview_name: 'Emma',
  });'''

# 6단계: Start 버튼 비활성화
start_pattern = r'const startButton = document\.getElementById\(\'start-button\'\);[\s\S]*?(?=const destroyButton)'
safe_start = '''const startButton = document.getElementById('start-button');
if (startButton) {
  startButton.onclick = () => {};
}'''

# 7단계: D-ID 응답 차단
dc_pattern = r'document\.getElementById\(\'msgHistory\'\)\.innerHTML \+= `<span>\$\{decodedMsg\}</span><br><br>`;'
dc_replacement = '// D-ID 응답 완전 차단'

textarea_pattern = r'document\.getElementById\(\'textArea\'\)\.value = decodedMsg;'
textarea_replacement = '// textArea에 D-ID 응답 입력 차단'

# 8단계: dc.onmessage 수정 - stream/done 처리 추가
dc_onmessage_pattern = r'dc\.onmessage = \(event\) => \{[\s\S]*?\};'
dc_onmessage_replacement = '''dc.onmessage = (event) => {
  let msg = event.data;
  let msgType = 'chat/answer:';
  if (msg.includes(msgType)) {
    msg = decodeURIComponent(msg.replace(msgType, ''));
    console.log(msg);
    decodedMsg = msg;
    return decodedMsg;
  }
  if (msg.includes('stream/started')) {
    console.log(msg);
    updateStatus('🎤 Emma가 말하는 중...', true);
  } else if (msg.includes('stream/done')) {
    console.log('TTS 완료 감지:', msg);
    isTTSSpeaking = false;
    updateStatus('✅ 음성 대기 중', false);

    // 1초 후 자막 숨기고 STT 재시작
    setTimeout(() => {
      hideSubtitle();
      console.log("stream/done 후 STT 재시작");
      startSTT();
    }, 1000);
  } else {
    console.log(msg);
  }
};'''

# 9단계: 모든 수정 적용
new_js = improved_stt_code + js_content
new_js = re.sub(agent_pattern, simple_agent, new_js)
new_js = re.sub(start_pattern, safe_start, new_js)
new_js = re.sub(dc_pattern, dc_replacement, new_js)
new_js = re.sub(textarea_pattern, textarea_replacement, new_js)
new_js = re.sub(dc_onmessage_pattern, dc_onmessage_replacement, new_js)

# 10단계: 저장
with open('agents-client-api.js', 'w', encoding='utf-8') as f:
    f.write(new_js)

print("🎉 완전 자동화 음성 대화 시스템이 완성되었습니다!")
print()
print("✅ 주요 기능:")
print("   🔄 완전 자동 연결 (Agent 생성 → Connect)")
print("   🎤 자동 음성 인식 (버튼 없음)")
print("   💬 실시간 자막 표시 (사용자/Emma 구분)")
print("   🔁 연속 대화 (TTS 종료 후 자동 STT 재시작)")
print("   🎨 깔끔한 UI (불필요한 요소 모두 숨김)")
print("   📍 우상단 상태창")
print()
print("🎯 사용법:")
print("   1. 페이지 로드하면 자동으로 Emma와 연결")
print("   2. 말하기만 하면 자동으로 대화 시작")
print("   3. 2초 침묵 시 자동으로 AI 응답")
print("   4. Emma 응답 완료 후 자동으로 다시 대기")
print()
print("💡 Emma의 얼굴과 자막만 보이는 완전한 핸즈프리 AI 어시스턴트!")

🎉 완전 자동화 음성 대화 시스템이 완성되었습니다!

✅ 주요 기능:
   🔄 완전 자동 연결 (Agent 생성 → Connect)
   🎤 자동 음성 인식 (버튼 없음)
   💬 실시간 자막 표시 (사용자/Emma 구분)
   🔁 연속 대화 (TTS 종료 후 자동 STT 재시작)
   🎨 깔끔한 UI (불필요한 요소 모두 숨김)
   📍 우상단 상태창

🎯 사용법:
   1. 페이지 로드하면 자동으로 Emma와 연결
   2. 말하기만 하면 자동으로 대화 시작
   3. 2초 침묵 시 자동으로 AI 응답
   4. Emma 응답 완료 후 자동으로 다시 대기

💡 Emma의 얼굴과 자막만 보이는 완전한 핸즈프리 AI 어시스턴트!


## 상태창과 아바타, 음성인식 버튼과 챗 히스토리 있는 버전

In [ ]:
import shutil
import re

# 1단계: 백업 복구
shutil.copy('agents-client-api.js.backup', 'agents-client-api.js')
shutil.copy('index-agents.html.backup', 'index-agents.html')

# 2단계: 원본 읽기
with open('agents-client-api.js', 'r', encoding='utf-8') as f:
    js_content = f.read()

# 3단계: 개선된 STT 코드 + 상태창 + 자동 연결 + UI 숨김
improved_stt_code = f'''// 상태 메시지창 생성 및 관리
function createStatusWindow() {{
  const existingStatus = document.getElementById('custom-status-window');
  if (existingStatus) {{
    existingStatus.remove();
  }}

  const statusWindow = document.createElement('div');
  statusWindow.id = 'custom-status-window';
  statusWindow.style.position = 'fixed';
  statusWindow.style.top = '20px';
  statusWindow.style.right = '20px';  // 우상단으로 다시 변경
  statusWindow.style.width = '280px';  // 조금 더 작게
  statusWindow.style.background = 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)';
  statusWindow.style.color = 'white';
  statusWindow.style.padding = '12px';  // 패딩 줄임
  statusWindow.style.borderRadius = '10px';
  statusWindow.style.boxShadow = '0 4px 15px rgba(0,0,0,0.2)';
  statusWindow.style.zIndex = '10000';
  statusWindow.style.fontFamily = 'Segoe UI, Tahoma, Geneva, Verdana, sans-serif';
  statusWindow.style.fontSize = '13px';  // 폰트 조금 작게
  statusWindow.style.lineHeight = '1.4';
  statusWindow.style.border = '2px solid rgba(255,255,255,0.2)';

  const statusTitle = document.createElement('div');
  statusTitle.style.fontWeight = 'bold';
  statusTitle.style.fontSize = '15px';  // 제목도 조금 작게
  statusTitle.style.marginBottom = '8px';  // 간격 줄임
  statusTitle.style.textAlign = 'center';
  statusTitle.style.borderBottom = '1px solid rgba(255,255,255,0.3)';
  statusTitle.style.paddingBottom = '6px';  // 패딩 줄임
  statusTitle.textContent = '🤖 Emma AI 상태';

  const statusContent = document.createElement('div');
  statusContent.id = 'status-content';
  statusContent.style.minHeight = '40px';  // 높이 줄임
  statusContent.style.display = 'flex';
  statusContent.style.alignItems = 'center';
  statusContent.style.justifyContent = 'center';

  statusWindow.appendChild(statusTitle);
  statusWindow.appendChild(statusContent);
  document.body.appendChild(statusWindow);

  return statusContent;
}}

function updateStatus(message, isLoading = false) {{
  const statusContent = document.getElementById('status-content');
  if (!statusContent) return;

  if (isLoading) {{
    const spinnerHtml = '<span style="display:inline-block;width:12px;height:12px;border:2px solid rgba(255,255,255,0.3);border-top:2px solid white;border-radius:50%;animation:spin 1s linear infinite;margin-right:8px;"></span>';
    const styleHtml = '<style>@keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}</style>';
    statusContent.innerHTML = '<div style="text-align: center;">' + spinnerHtml + message + '</div>' + styleHtml;
  }} else {{
    statusContent.innerHTML = '<div style="text-align: center;">' + message + '</div>';
  }}
}}

// UI 요소들 숨기기 함수 (대화창 보존)
function hideUIElements() {{
  // 메인 status div 강제 숨기기
  const statusDiv = document.getElementById('status');
  if (statusDiv) {{
    statusDiv.style.display = 'none !important';
    statusDiv.style.visibility = 'hidden';
    statusDiv.style.opacity = '0';
    statusDiv.style.position = 'absolute';
    statusDiv.style.left = '-9999px';
    console.log('Main status div 완전 숨김');
  }}

  // Chat History는 보이게 유지
  const chatDiv = document.querySelector('.chat');
  if (chatDiv) {{
    chatDiv.style.display = 'block';
    chatDiv.style.visibility = 'visible';

    // Chat History 제목만 숨기기
    const chatTitle = chatDiv.querySelector('h4');
    if (chatTitle) {{
      chatTitle.style.display = 'none';
    }}
    console.log('Chat history 표시 (제목만 숨김)');
  }}

  // Type your message 영역은 숨기되, textArea는 보이게
  const messageH3 = document.querySelector('h3');
  if (messageH3 && messageH3.textContent.includes('Type your message here')) {{
    messageH3.style.display = 'none'; // 제목만 숨기기
  }}

  // textarea와 메시지 입력 영역 보이게 유지
  const textarea = document.getElementById('textArea');
  if (textarea) {{
    textarea.style.display = 'block';
    textarea.style.visibility = 'visible';
    // 부모 div도 보이게
    if (textarea.parentElement) {{
      textarea.parentElement.style.display = 'block';
      textarea.parentElement.style.visibility = 'visible';
    }}
  }}

  // 모든 h4 제목 중 Agent, WebRTC 관련만 숨기기
  const h4Elements = document.querySelectorAll('h4');
  h4Elements.forEach(h4 => {{
    if (h4.textContent.includes('Agent Status') ||
        h4.textContent.includes('WebRTC Connection Status')) {{
      h4.style.display = 'none';
    }}
    // Chat History 제목은 이미 위에서 처리함
  }});

  // 상태 관련 버튼들과 Send 버튼 숨기기
  const hideButtons = ['agents-button', 'connect-button', 'destroy-button', 'start-button'];
  hideButtons.forEach(id => {{
    const btn = document.getElementById(id);
    if (btn) {{
      btn.style.display = 'none';
      console.log(id + ' 버튼 숨김');
    }}
  }});

  // Send 버튼 특별히 숨기기 (여러 방법으로)
  const sendBtn = document.getElementById('start-button');
  if (sendBtn) {{
    sendBtn.style.display = 'none';
    sendBtn.style.visibility = 'hidden';
    sendBtn.style.opacity = '0';
    sendBtn.style.position = 'absolute';
    sendBtn.style.left = '-9999px';
    console.log('Send 버튼 완전 숨김');
  }}

  // 모든 라벨과 상태 정보 숨기기
  const labels = document.querySelectorAll('label');
  labels.forEach(label => {{
    if (label.id && (label.id.includes('status') || label.id.includes('agent') || label.id.includes('chat'))) {{
      label.style.display = 'none';
    }}
  }});

  // br 태그들 정리
  const brs = document.querySelectorAll('br');
  brs.forEach(br => {{
    if (br.parentElement && br.parentElement.id === 'status') {{
      br.style.display = 'none';
    }}
  }});

  console.log('UI 요소들 선택적 숨김 처리 완료');
}}

// 자동 연결 및 시작 함수
async function autoConnectAndStart() {{
  updateStatus('시스템 초기화 중...', true);
  console.log('자동 연결 시작...');

  try {{
    // 1. agents 버튼 클릭
    const agentsBtn = document.getElementById('agents-button');
    if (agentsBtn && !agentsBtn.disabled) {{
      updateStatus('Emma AI 에이전트 생성 중...', true);
      agentsBtn.click();
      console.log('Agents 생성 중...');

      // agents 완료까지 대기
      await waitForElementEnabled('connect-button', 60000);
      console.log('Agents 생성 완료');

      // Knowledge 처리 완료 대기 (더 긴 시간)
      await new Promise(resolve => setTimeout(resolve, 5000));

      // 2. connect 버튼 적극적으로 클릭
      updateStatus('Emma와 연결하는 중...', true);

      let connectSuccess = false;
      for (let attempt = 1; attempt <= 5; attempt++) {{
        console.log(`Connect 시도 ${{attempt}}/5`);

        const connectBtn = document.getElementById('connect-button');
        console.log('Connect 버튼 상태:', connectBtn ? 'exist' : 'not found', connectBtn ? (connectBtn.disabled ? 'disabled' : 'enabled') : 'N/A');

        if (connectBtn) {{
          // 버튼 강제 활성화
          connectBtn.disabled = false;
          connectBtn.click();

          console.log(`Connect 버튼 클릭 완료 (시도 ${{attempt}})`);

          // 연결 확인 대기
          try {{
            await waitForConnection(15000); // 15초 대기
            connectSuccess = true;
            console.log('연결 성공!');
            break;
          }} catch (error) {{
            console.log(`연결 시도 ${{attempt}} 실패:`, error.message);
            if (attempt < 5) {{
              await new Promise(resolve => setTimeout(resolve, 3000)); // 3초 대기 후 재시도
            }}
          }}
        }} else {{
          console.error('Connect 버튼을 찾을 수 없음');
          break;
        }}
      }}

      if (connectSuccess) {{
        updateStatus('✅ 연결 완료! 음성인식 준비됨', false);
        console.log('자동 연결 완료!');
        return true;
      }} else {{
        updateStatus('❌ 연결 실패 - Connect 버튼을 수동으로 클릭하세요', false);
        console.error('모든 연결 시도 실패');
        return false;
      }}
    }} else {{
      console.error('Agents 버튼을 찾을 수 없음');
      updateStatus('❌ Agents 버튼 오류', false);
      return false;
    }}
  }} catch (error) {{
    console.error('자동 연결 중 오류:', error);
    updateStatus('❌ 자동연결 실패', false);
    return false;
  }}
}}

// 연결 완료까지 대기하는 함수 (streamId 확실히 확인)
function waitForConnection(timeout = 30000) {{
  return new Promise((resolve, reject) => {{
    const startTime = Date.now();

    function checkConnection() {{
      // 여러 방법으로 연결 상태 확인
      const connectionChecks = [
        // 1. streamId 확인
        () => {{
          const isValid = typeof streamId !== 'undefined' && streamId !== null && streamId !== '';
          if (isValid) console.log('StreamId 확인됨:', streamId);
          return isValid;
        }},

        // 2. video element src 확인 (실제 스트림)
        () => {{
          const videoEl = document.getElementById('video-element');
          const hasStream = videoEl && videoEl.src &&
                           !videoEl.src.includes('emma_idle.mp4') &&
                           videoEl.src.includes('blob:');
          if (hasStream) console.log('Video stream 확인됨:', videoEl.src);
          return hasStream;
        }},

        // 3. WebRTC 상태 확인
        () => {{
          const peerLabel = document.getElementById('peer-status-label');
          const isConnected = peerLabel && (peerLabel.textContent.includes('connected') || peerLabel.className.includes('connected'));
          if (isConnected) console.log('Peer connection 확인됨');
          return isConnected;
        }},

        // 4. ICE 상태 확인
        () => {{
          const iceLabel = document.getElementById('ice-status-label');
          const isConnected = iceLabel && (iceLabel.textContent.includes('connected') || iceLabel.className.includes('connected'));
          if (isConnected) console.log('ICE connection 확인됨');
          return isConnected;
        }}
      ];

      // 하나라도 연결된 것으로 판단되면 성공
      const isConnected = connectionChecks.some(check => {{
        try {{
          return check();
        }} catch (e) {{
          return false;
        }}
      }});

      if (isConnected) {{
        console.log('연결 상태 확인됨');

        // streamId가 없다면 조금 더 기다리기
        if (typeof streamId === 'undefined' || streamId === null || streamId === '') {{
          console.log('StreamId 생성 대기 중...');
          setTimeout(checkConnection, 2000); // 2초 더 대기
          return;
        }}

        resolve();
      }} else {{
        const elapsed = Math.floor((Date.now() - startTime) / 1000);
        if (elapsed % 3 === 0 && elapsed > 0) {{
          console.log(`연결 대기 중... ${{elapsed}}초`);
          updateStatus(`연결 중... (${{elapsed}}초)`, true);
        }}

        if (Date.now() - startTime > timeout) {{
          reject(new Error('Connection timeout'));
        }} else {{
          setTimeout(checkConnection, 1000);
        }}
      }}
    }}

    checkConnection();
  }});
}}

// 버튼이 활성화될 때까지 대기하는 함수
function waitForElementEnabled(elementId, timeout = 30000) {{
  return new Promise((resolve, reject) => {{
    const startTime = Date.now();

    function checkElement() {{
      const element = document.getElementById(elementId);
      if (element && !element.disabled) {{
        resolve(element);
      }} else {{
        const elapsed = Math.floor((Date.now() - startTime) / 1000);
        if (elapsed % 5 === 0 && elapsed > 0) {{
          updateStatus(`${{elementId}} 활성화 대기 중... (${{elapsed}}초)`, true);
        }}

        if (Date.now() - startTime > timeout) {{
          reject(new Error(`Timeout waiting for ${{elementId}}`));
        }} else {{
          setTimeout(checkElement, 500);
        }}
      }}
    }}

    checkElement();
  }});
}}

// OpenAI 직접 호출
async function callOpenAI(message) {{
  updateStatus('AI가 답변을 생각하는 중...', true);

  const response = await fetch('https://api.openai.com/v1/chat/completions', {{
    method: 'POST',
    headers: {{
      'Authorization': 'Bearer {openai_api_key}',
      'Content-Type': 'application/json'
    }},
    body: JSON.stringify({{
      model: 'gpt-4o-mini',
      messages: [{{
        role: 'system',
        content: '당신은 Emma입니다. 항상 한국어로 대답하세요. 답변 시 "*"나 마크업 기능은 사용하지 말아주세요.'
      }}, {{
        role: 'user',
        content: message
      }}]
    }})
  }});
  const data = await response.json();
  updateStatus('음성으로 답변하는 중...', true);
  return data.choices[0].message.content;
}}

// 개선된 STT 생성 함수 (끊김 문제 해결)
function forceCreateSTT() {{

  const sendBtn = document.getElementById('start-button');
  if (!sendBtn) {{
    return;
  }}

  // 기존 STT 버튼이 있다면 제거하고 새로 생성
  let oldSttBtn = document.getElementById('stt-button');
  if (oldSttBtn) {{
    oldSttBtn.remove();
  }}

  // 새 STT 버튼 생성
  const sttBtn = document.createElement('button');
  sttBtn.id = 'stt-button';
  sttBtn.textContent = '🎤 음성인식';
  sttBtn.style.cssText = 'background:#28a745; color:white; border:none; border-radius:8px; padding:10px; margin-right:10px; cursor:pointer; font-size:14px;';

  sendBtn.parentNode.insertBefore(sttBtn, sendBtn);

  // 음성인식 객체 생성
  let recognition = null;
  let isRecording = false;
  let finalTranscript = '';

  function initRecognition() {{
    recognition = new (window.SpeechRecognition || window.webkitSpeechRecognition)();
    recognition.lang = 'ko-KR';
    recognition.continuous = true;  // 연속 인식
    recognition.interimResults = true;  // 중간 결과도 받기
    recognition.maxAlternatives = 1;

    // 음성인식 결과 처리
    recognition.onresult = (event) => {{
      let interimTranscript = '';

      for (let i = event.resultIndex; i < event.results.length; i++) {{
        const transcript = event.results[i][0].transcript;

        if (event.results[i].isFinal) {{
          finalTranscript += transcript;
        }} else {{
          interimTranscript += transcript;
        }}
      }}

      // 실시간으로 텍스트 영역에 표시
      const textArea = document.getElementById('textArea');
      textArea.value = finalTranscript + interimTranscript;
    }};

    // 음성인식 시작
    recognition.onstart = () => {{
      isRecording = true;
      sttBtn.textContent = '🎙️ 말하세요... (클릭하면 종료)';
      sttBtn.style.backgroundColor = '#dc3545';
      updateStatus('🎤 음성 인식 중...', true);
    }};

    // 음성인식 종료
    recognition.onend = () => {{
      isRecording = false;
      sttBtn.textContent = '🎤 음성인식';
      sttBtn.style.backgroundColor = '#28a745';

      // 최종 결과가 있으면 텍스트 영역에 설정하고 자동 전송
      if (finalTranscript.trim()) {{
        document.getElementById('textArea').value = finalTranscript.trim();
        updateStatus('메시지 전송 준비 중...', true);

        // 1초 후 자동으로 Send 버튼 클릭
        setTimeout(() => {{
          const sendBtn = document.getElementById('start-button');
          if (sendBtn) {{
            sendBtn.click();
          }}
        }}, 1000);
      }} else {{
        updateStatus('✅ 음성인식 준비됨', false);
      }}
    }};

  }}

  // STT 버튼 클릭 이벤트
  sttBtn.onclick = () => {{
    if (!isRecording) {{
      // 음성인식 시작
      finalTranscript = '';
      initRecognition();
      recognition.start();
    }} else {{
      // 음성인식 중지
      recognition.stop();
    }}
  }};
}}

// 페이지 로드 시 자동 실행 (순서 조정)
document.addEventListener('DOMContentLoaded', () => {{
  setTimeout(async () => {{
    createStatusWindow();
    updateStatus('시스템 초기화 중...', true);

    // 먼저 자동 연결 시도
    const success = await autoConnectAndStart();

    // 연결 완료 후 UI 숨김
    setTimeout(() => {{
      hideUIElements();
    }}, 2000);

    if (success) {{
      forceCreateSTT();
    }} else {{
      updateStatus('❌ 연결 실패 - Connect 버튼을 수동으로 클릭하세요', false);

      // 실패 시 버튼들을 다시 보이게 함
      setTimeout(() => {{
        const connectBtn = document.getElementById('connect-button');
        const agentsBtn = document.getElementById('agents-button');
        if (connectBtn) connectBtn.style.display = 'inline-block';
        if (agentsBtn) agentsBtn.style.display = 'inline-block';

        updateStatus('⚠️ 수동으로 Connect 버튼을 클릭하세요', false);
      }}, 1000);
    }}
  }}, 2000);
}});

// 추가 보장 (연결 확인 후 UI 정리)
window.addEventListener('load', () => {{
  setTimeout(() => {{
    // 상태창이 없으면 생성
    if (!document.getElementById('custom-status-window')) {{
      createStatusWindow();
      updateStatus('✅ 음성인식 준비됨', false);
    }}

    // 연결 상태 확인
    const isConnected = typeof streamId !== 'undefined' && streamId !== null && streamId !== '';
    const videoElement = document.getElementById('video-element');
    const hasVideoSrc = videoElement && videoElement.src && videoElement.src !== '';

    if (isConnected || hasVideoSrc) {{
      // 연결됨 - UI 완전 숨김
      hideUIElements();
      updateStatus('✅ 연결 완료! 음성인식 준비됨', false);
    }} else {{
      // 연결 안됨 - 필요한 버튼만 표시
      const connectBtn = document.getElementById('connect-button');
      if (connectBtn) {{
        connectBtn.style.display = 'inline-block';
        connectBtn.style.visibility = 'visible';
        connectBtn.style.position = 'static';
        connectBtn.style.left = 'auto';
      }}
      updateStatus('⚠️ Connect 버튼을 클릭하세요', false);
    }}

    forceCreateSTT();
  }}, 3000);
}});

// 최종 보장 (스마트 처리)
setTimeout(() => {{
  // 상태창 확인 및 생성
  if (!document.getElementById('custom-status-window')) {{
    createStatusWindow();
  }}

  // 연결 상태에 따른 처리
  const videoElement = document.getElementById('video-element');
  const hasVideo = videoElement && videoElement.src && !videoElement.src.includes('emma_idle.mp4');
  const isConnected = typeof streamId !== 'undefined' && streamId !== null && streamId !== '';

  if (hasVideo || isConnected) {{
    // 성공적으로 연결된 경우
    hideUIElements();
    updateStatus('✅ 음성 연결 완료!', false);
    console.log('연결 성공 확인 - UI 완전 숨김');
  }} else {{
    // 연결 실패한 경우 - Connect 버튼만 복원
    const statusDiv = document.getElementById('status');
    const connectBtn = document.getElementById('connect-button');

    if (statusDiv && connectBtn) {{
      // status div를 부분적으로 복원
      statusDiv.style.display = 'block';
      statusDiv.style.visibility = 'visible';
      statusDiv.style.position = 'static';
      statusDiv.style.left = 'auto';
      statusDiv.style.opacity = '1';

      // Connect 버튼만 보이게
      connectBtn.style.display = 'inline-block';

      updateStatus('⚠️ Connect 버튼을 수동으로 클릭하세요', false);
      console.log('연결 실패 - Connect 버튼 복원');
    }} else {{
      updateStatus('❌ 페이지를 새로고침하세요', false);
    }}
  }}

  forceCreateSTT();
}}, 8000);

'''

# 4단계: Agent 설정 (간단하게)
agent_pattern = r'const createAgent = await axios\.post\(\'/agents\', \{[\s\S]*?\}\);'
simple_agent = '''const createAgent = await axios.post('/agents', {
    presenter: {
      type: 'talk',
      voice: {
        type: 'microsoft',
        voice_id: 'ko-KR-SunHiNeural',
      },
      thumbnail: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
      source_url: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
    },
    preview_name: 'Emma',
  });'''

# 5단계: Send 버튼 교체 (기존 방식 + 상태 업데이트)
start_pattern = r'const startButton = document\.getElementById\(\'start-button\'\);[\s\S]*?(?=const destroyButton)'
simple_start = '''const startButton = document.getElementById('start-button');
startButton.onclick = async () => {
  const txtAreaValue = document.getElementById('textArea').value;
  document.getElementById('msgHistory').innerHTML += `<span style='opacity:0.5'><u>User:</u> ${txtAreaValue}</span><br>`;
  document.getElementById('textArea').value = '';

  const aiResponse = await callOpenAI(txtAreaValue);
  document.getElementById('msgHistory').innerHTML += `<span>${aiResponse}</span><br><br>`;

  await fetchWithRetries(`${DID_API.url}/talks/streams/${streamId}`, {
    method: 'POST',
    headers: {
      Authorization: `Basic ${DID_API.key}`,
      'Content-Type': 'application/json',
    },
    body: JSON.stringify({
      script: {
        type: 'text',
        input: aiResponse,
        provider: {
          type: 'microsoft',
          voice_id: 'ko-KR-SunHiNeural'
        }
      },
      session_id: sessionId,
    }),
  });

  updateStatus('✅ 음성인식 준비됨', false);
};

'''

# 6단계: D-ID 응답 완전 차단 (사용자 입력창에 표시되지 않도록)
dc_pattern = r'document\.getElementById\(\'msgHistory\'\)\.innerHTML \+= `<span>\$\{decodedMsg\}</span><br><br>`;'
dc_replacement = '// D-ID 응답 완전 차단'

# 추가: D-ID 메시지가 textArea에 들어가는 것도 차단
textarea_pattern = r'document\.getElementById\(\'textArea\'\)\.value = decodedMsg;'
textarea_replacement = '// textArea에 D-ID 응답 입력 차단'

# 7단계: 모든 수정 적용
new_js = improved_stt_code + js_content
new_js = re.sub(agent_pattern, simple_agent, new_js)
new_js = re.sub(start_pattern, simple_start, new_js)
new_js = re.sub(dc_pattern, dc_replacement, new_js)
new_js = re.sub(textarea_pattern, textarea_replacement, new_js)

# 8단계: 저장
with open('agents-client-api.js', 'w', encoding='utf-8') as f:
    f.write(new_js)

## 모든 UI 요소가 다 있는 버전

In [ ]:
# 1단계: 백업 복구
shutil.copy('agents-client-api.js.backup', 'agents-client-api.js')
shutil.copy('index-agents.html.backup', 'index-agents.html')

# 2단계: 원본 읽기
with open('agents-client-api.js', 'r', encoding='utf-8') as f:
    js_content = f.read()

# 3단계: 개선된 STT 코드 (끊김 문제 해결)
improved_stt_code = f'''// OpenAI 직접 호출
async function callOpenAI(message) {{
  const response = await fetch('https://api.openai.com/v1/chat/completions', {{
    method: 'POST',
    headers: {{
      'Authorization': 'Bearer {openai_api_key}',
      'Content-Type': 'application/json'
    }},
    body: JSON.stringify({{
      model: 'gpt-4o-mini',
      messages: [{{
        role: 'system',
        content: '당신은 Emma입니다. 항상 한국어로 대답하세요. 답변 시 "*"나 마크업 기능은 사용하지 말아주세요.'
      }}, {{
        role: 'user',
        content: message
      }}]
    }})
  }});
  const data = await response.json();
  return data.choices[0].message.content;
}}

// 개선된 STT 생성 함수 (끊김 문제 해결)
function forceCreateSTT() {{

  const sendBtn = document.getElementById('start-button');
  if (!sendBtn) {{
    return;
  }}

  // 기존 STT 버튼이 있다면 제거하고 새로 생성
  let oldSttBtn = document.getElementById('stt-button');
  if (oldSttBtn) {{
    oldSttBtn.remove();
  }}

  // 새 STT 버튼 생성
  const sttBtn = document.createElement('button');
  sttBtn.id = 'stt-button';
  sttBtn.textContent = '🎤 음성인식';
  sttBtn.style.cssText = 'background:#28a745; color:white; border:none; border-radius:8px; padding:10px; margin-right:10px; cursor:pointer; font-size:14px;';

  sendBtn.parentNode.insertBefore(sttBtn, sendBtn);

  // 음성인식 객체 생성
  let recognition = null;
  let isRecording = false;
  let finalTranscript = '';

  function initRecognition() {{
    recognition = new (window.SpeechRecognition || window.webkitSpeechRecognition)();
    recognition.lang = 'ko-KR';
    recognition.continuous = true;  // 연속 인식
    recognition.interimResults = true;  // 중간 결과도 받기
    recognition.maxAlternatives = 1;

    // 음성인식 결과 처리
    recognition.onresult = (event) => {{
      let interimTranscript = '';

      for (let i = event.resultIndex; i < event.results.length; i++) {{
        const transcript = event.results[i][0].transcript;

        if (event.results[i].isFinal) {{
          finalTranscript += transcript;
        }} else {{
          interimTranscript += transcript;
        }}
      }}

      // 실시간으로 텍스트 영역에 표시
      const textArea = document.getElementById('textArea');
      textArea.value = finalTranscript + interimTranscript;
    }};

    // 음성인식 시작
    recognition.onstart = () => {{
      isRecording = true;
      sttBtn.textContent = '🎙️ 말하세요... (클릭하면 종료)';
      sttBtn.style.backgroundColor = '#dc3545';
    }};

    // 음성인식 종료
    recognition.onend = () => {{
      isRecording = false;
      sttBtn.textContent = '🎤 음성인식';
      sttBtn.style.backgroundColor = '#28a745';

      // 최종 결과가 있으면 텍스트 영역에 설정하고 자동 전송
      if (finalTranscript.trim()) {{
        document.getElementById('textArea').value = finalTranscript.trim();

        // 1초 후 자동으로 Send 버튼 클릭
        setTimeout(() => {{
          const sendBtn = document.getElementById('start-button');
          if (sendBtn) {{
            sendBtn.click();
          }}
        }}, 1000);
      }}
    }};


  }}

  // STT 버튼 클릭 이벤트
  sttBtn.onclick = () => {{
    if (!isRecording) {{
      // 음성인식 시작
      finalTranscript = '';
      initRecognition();
      recognition.start();
    }} else {{
      // 음성인식 중지
      recognition.stop();
    }}
  }};
}}

// 페이지 로드 시 STT 생성
document.addEventListener('DOMContentLoaded', () => {{
  setTimeout(forceCreateSTT, 1000);
}});

// 추가 보장
window.addEventListener('load', () => {{
  setTimeout(forceCreateSTT, 2000);
}});

// 최종 보장
setTimeout(() => {{
  forceCreateSTT();
}}, 5000);

'''

# 4단계: Agent 설정 (간단하게)
agent_pattern = r'const createAgent = await axios\.post\(\'/agents\', \{[\s\S]*?\}\);'
simple_agent = '''const createAgent = await axios.post('/agents', {
    presenter: {
      type: 'talk',
      voice: {
        type: 'microsoft',
        voice_id: 'ko-KR-SunHiNeural',
      },
      thumbnail: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
      source_url: 'https://create-images-results.d-id.com/DefaultPresenters/Emma_f/v1_image.jpeg',
    },
    preview_name: 'Emma',
  });'''

# 5단계: Send 버튼 교체
start_pattern = r'const startButton = document\.getElementById\(\'start-button\'\);[\s\S]*?(?=const destroyButton)'
simple_start = '''const startButton = document.getElementById('start-button');
startButton.onclick = async () => {
  const txtAreaValue = document.getElementById('textArea').value;
  document.getElementById('msgHistory').innerHTML += `<span style='opacity:0.5'><u>User:</u> ${txtAreaValue}</span><br>`;
  document.getElementById('textArea').value = '';

  const aiResponse = await callOpenAI(txtAreaValue);
  document.getElementById('msgHistory').innerHTML += `<span>${aiResponse}</span><br><br>`;

  await fetchWithRetries(`${DID_API.url}/talks/streams/${streamId}`, {
    method: 'POST',
    headers: {
      Authorization: `Basic ${DID_API.key}`,
      'Content-Type': 'application/json',
    },
    body: JSON.stringify({
      script: {
        type: 'text',
        input: aiResponse,
        provider: {
          type: 'microsoft',
          voice_id: 'ko-KR-SunHiNeural'
        }
      },
      session_id: sessionId,
    }),
  });
};

'''

# 6단계: D-ID 응답 완전 차단 (사용자 입력창에 표시되지 않도록)
dc_pattern = r'document\.getElementById\(\'msgHistory\'\)\.innerHTML \+= `<span>\$\{decodedMsg\}</span><br><br>`;'
dc_replacement = '// D-ID 응답 완전 차단'

# 추가: D-ID 메시지가 textArea에 들어가는 것도 차단
textarea_pattern = r'document\.getElementById\(\'textArea\'\)\.value = decodedMsg;'
textarea_replacement = '// textArea에 D-ID 응답 입력 차단'

# 7단계: 모든 수정 적용
new_js = improved_stt_code + js_content
new_js = re.sub(agent_pattern, simple_agent, new_js)
new_js = re.sub(start_pattern, simple_start, new_js)
new_js = re.sub(dc_pattern, dc_replacement, new_js)
new_js = re.sub(textarea_pattern, textarea_replacement, new_js)

# 8단계: 저장
with open('agents-client-api.js', 'w', encoding='utf-8') as f:
    f.write(new_js)

`F5 클릭하여 화면 고침하고 Create new Agent with Knowledge 클릭부터 다시 시작하면 됩니다.`